# Pre-processing Nebraska Allocation data for WaDEQA upload.
Date Updated: 08/26/2020
Purpose:  To pre-process the Nebraska data into one master file for simple DataFrame creation and extraction

Data was provdied by NeDWR as a shapefile via email, which was exported to a csv.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input Files
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Nebraska/WaterAllocation/RawInputData"
os.chdir(workingDir)
inputFile = "NebraskaSurfaceWaterAPIData.xlsx"

In [3]:
#Dataframe creation
df = pd.DataFrame() #Output
df = pd.read_excel(inputFile) #Input
df

,RightID,RightStatus,RightUse,IrrigationProject,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,ProGrant = Acres *(1/Rate),CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,Long,Lat,County
0,4105,Active,IR,NaN,A-10003,NWNE S34-T2-R8E,176836,1.0,1.0,1D,NaN,Pump,NaN,Wolf Creek,2.64,CFS,1184.0,70.0,185.0,NaN,Gage,Lower Big Blue,1.027020e+11,1961-12-21T00:00:00,NaN,1962-07-16T00:00:00,Open,2016-04-16T00:00:00,Open,Natural Flow (Reminder),-96.514694,40.094689,Gage
1,6155,Active,IR,NaN,A-10009,NENE S25-T17-R7W,188876,1.0,1.0,2A,NaN,Pump,NaN,Timber Creek,0.77,CFS,345.0,70.0,54.0,NaN,Nance,Lower Loup,1.021001e+11,1962-01-17T00:00:00,NaN,NaN,Open,2016-04-28T00:00:00,Open,Natural Flow (Reminder),-98.078058,41.411544,Nance
2,7967,Active,IR,NaN,A-10013,NENE S14-T34-R36W,188035,1.0,1.0,2C,NaN,Wickman Canals,NaN,Bear Creek,0.58,CFS,260.0,70.0,40.6,NaN,Cherry,Middle Niobrara,1.015000e+11,1962-02-13T00:00:00,NaN,NaN,Open,2016-04-18T00:00:00,Open,Natural Flow (Reminder),-101.534059,42.917162,Cherry
3,2410,Active,IR,NaN,A-10014,NW S25-T5-R3W,177345,1.0,1.0,1C,NaN,Pump,NaN,"Sandy Creek, Dry, Trib. To",0.74,CFS,332.0,70.0,52.0,NaN,Fillmore,Little Blue,1.027021e+11,1962-02-13T00:00:00,NaN,NaN,Open,2016-04-15T00:00:00,Open,Natural Flow (Reminder),-97.613026,40.370681,Fillmore
4,8464,Active,SO,NaN,A-10020,NENW S4-T32-R11W,190897,1.0,1.0,2C,NaN,Witherwax Canal,NaN,Witherwax Reservoir,14.00,AF,0.0,NaN,32.1,NaN,Holt,Lower Niobrara,1.015001e+11,1961-02-06T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,-98.611769,42.784548,Holt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,13428,Removed,Removed,Removed,Removed,Removed,194054,NaN,NaN,Removed,Removed,Removed,Removed,Removed,NaN,Removed,NaN,NaN,NaN,NaN,Removed,Removed,NaN,NaN,NaN,NaN,Removed,NaN,Removed,Removed,NaN,NaN,NaN
8785,13429,Removed,Removed,Removed,Removed,Removed,193855,NaN,NaN,Removed,Removed,Removed,Removed,Removed,NaN,Removed,NaN,NaN,NaN,NaN,Removed,Removed,NaN,NaN,NaN,NaN,Removed,NaN,Removed,Removed,NaN,NaN,NaN
8786,13431,Removed,Removed,Removed,Removed,Removed,193856,NaN,NaN,Removed,Removed,Removed,Removed,Removed,NaN,Removed,NaN,NaN,NaN,NaN,Removed,Removed,NaN,NaN,NaN,NaN,Removed,NaN,Removed,Removed,NaN,NaN,NaN
8787,13432,Removed,Removed,Removed,Removed,Removed,194058,NaN,NaN,Removed,Removed,Removed,Removed,Removed,NaN,Removed,NaN,NaN,NaN,NaN,Removed,Removed,NaN,NaN,NaN,NaN,Removed,NaN,Removed,Removed,NaN,NaN,NaN


In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)

RightID                         int64
RightStatus                    object
RightUse                       object
IrrigationProject              object
ApplicationNumber              object
Location                       object
PointOfDiversionID              int64
PrimaryIndicator              float64
PointofDiversionStatus        float64
WaterDivision                  object
FieldOffice                    object
CarrierA                       object
CarrierB                       object
SourceName                     object
ProGrant                      float64
Units                          object
GPM                           float64
Rate                          float64
CurrentTotalAcres             float64
ProGrant = Acres *(1/Rate)    float64
CountyName                     object
NRDName                        object
HUC12                         float64
PriorityDate                   object
BeneficialDate                 object
ApprovalDate                   object
Notice_Statu

In [5]:
#temp fix.
#dropping rows with null Allowcation Amounts
dropIndex = df.loc[df['GPM'].isnull()].index

if len(dropIndex) > 0:
    df = df.drop(dropIndex)
    df = df.reset_index(drop=True)
df

,RightID,RightStatus,RightUse,IrrigationProject,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,ProGrant = Acres *(1/Rate),CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,Long,Lat,County
0,4105,Active,IR,NaN,A-10003,NWNE S34-T2-R8E,176836,1.0,1.0,1D,NaN,Pump,NaN,Wolf Creek,2.64,CFS,1184.0,70.0,185.0,NaN,Gage,Lower Big Blue,1.027020e+11,1961-12-21T00:00:00,NaN,1962-07-16T00:00:00,Open,2016-04-16T00:00:00,Open,Natural Flow (Reminder),-96.514694,40.094689,Gage
1,6155,Active,IR,NaN,A-10009,NENE S25-T17-R7W,188876,1.0,1.0,2A,NaN,Pump,NaN,Timber Creek,0.77,CFS,345.0,70.0,54.0,NaN,Nance,Lower Loup,1.021001e+11,1962-01-17T00:00:00,NaN,NaN,Open,2016-04-28T00:00:00,Open,Natural Flow (Reminder),-98.078058,41.411544,Nance
2,7967,Active,IR,NaN,A-10013,NENE S14-T34-R36W,188035,1.0,1.0,2C,NaN,Wickman Canals,NaN,Bear Creek,0.58,CFS,260.0,70.0,40.6,NaN,Cherry,Middle Niobrara,1.015000e+11,1962-02-13T00:00:00,NaN,NaN,Open,2016-04-18T00:00:00,Open,Natural Flow (Reminder),-101.534059,42.917162,Cherry
3,2410,Active,IR,NaN,A-10014,NW S25-T5-R3W,177345,1.0,1.0,1C,NaN,Pump,NaN,"Sandy Creek, Dry, Trib. To",0.74,CFS,332.0,70.0,52.0,NaN,Fillmore,Little Blue,1.027021e+11,1962-02-13T00:00:00,NaN,NaN,Open,2016-04-15T00:00:00,Open,Natural Flow (Reminder),-97.613026,40.370681,Fillmore
4,8464,Active,SO,NaN,A-10020,NENW S4-T32-R11W,190897,1.0,1.0,2C,NaN,Witherwax Canal,NaN,Witherwax Reservoir,14.00,AF,0.0,NaN,32.1,NaN,Holt,Lower Niobrara,1.015001e+11,1961-02-06T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,-98.611769,42.784548,Holt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8112,7784,Active,IR,NaN,D-980B,S25-T29-R56W,187139,1.0,1.0,2C,NaN,Mcginley-Stover N. & Cook Canal No. 2,NaN,Niobrara River,0.16,CFS,71.0,70.0,10.9,NaN,Sioux,Upper Niobrara-White,1.015000e+11,1891-05-31T00:00:00,NaN,1899-11-11T00:00:00,Regulating,2018-06-16T00:00:00,Regulating,Gage Height Posting,-103.842556,42.454258,Sioux
8113,7787,Active,IR,NaN,D-980R,SENW S1-T28-R56W,187136,1.0,1.0,2C,NaN,Pump,NaN,Niobrara River,0.52,CFS,233.0,70.0,36.1,NaN,Sioux,Upper Niobrara-White,1.015000e+11,1891-05-31T00:00:00,NaN,NaN,Open,2017-09-22T00:00:00,Open,Spencer Hydropower,-103.811240,42.430200,Sioux
8114,9099,Active,IR,NaN,D-981,S7-T33-R56W,187867,1.0,1.0,2E,NaN,Dout Brothers Canal,NaN,Jim Creek,0.64,CFS,287.0,70.0,45.0,NaN,Sioux,Upper Niobrara-White,1.012011e+11,1889-05-15T00:00:00,NaN,1900-02-09T00:00:00,Close,2017-09-13T00:00:00,Close,Riparian Administration,-103.957085,42.853727,Sioux
8115,9165,Active,IR,NaN,D-984,S22-T33-R54W,183608,1.0,1.0,2E,NaN,Homestead Canal,NaN,"Jim Creek, East, Trib. To",0.21,CFS,94.0,70.0,15.0,NaN,Sioux,Upper Niobrara-White,1.012011e+11,1890-05-31T00:00:00,NaN,1899-11-04T00:00:00,NaN,NaN,NaN,NaN,-103.662795,42.813928,Sioux


In [6]:
#Changing datatype of used date fields. 
df['PriorityDate'] = pd.to_datetime(df['PriorityDate'], errors = 'coerce')
df['PriorityDate'] = pd.to_datetime(df["PriorityDate"].dt.strftime('%m/%d/%Y'))

In [7]:
#Making Sure datatype of Long, Lat, Wrex, Irrigation are Float
def assignHUC12(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        outList = int(colrowValue)
    return outList

df['HUC12'] = df.apply(lambda row: assignHUC12(row['HUC12']), axis=1)
df['HUC12']

0       102702020604
1       102100100503
2       101500031608
3       102702060703
4       101500070504
            ...     
8112    101500020401
8113    101500020507
8114    101201080101
8115    101201080106
8116    101500020605
Name: HUC12, Length: 8117, dtype: int64

In [8]:
#Creating Benefcial Use from their code

NebraskaBenUseCodeDict = {
"CO" : "Cooling",
"DG" : "Dredge",
"DI" : "Domestic, Irrigation and Manufacturing",
"DO" : "Domestic",
"DS" : "Domestic Storage",
"FC" : "Fish Culture",
"FL" : "Flood Control",
"FW" : "Fish and Wildlife",
"IF" : "Instream Flow",
"IG" : "Induced Ground Water Recharge",
"IN" : "Intentional Underground Storage",
"IR" : "Irrigation from Natural Stream",
"IS" : "Irrigation and Storage (an appropriation approved for both uses)",
"IU" : "Irrigation and Incidental Underground Storage",
"MF" : "Manufacturing",
"ML" : "Maintain Level of a Lake",
"MU" : "Municipal",
"OU" : "Storage Use-only for irrigation from reservoir on lands not covered by natural flow appropriation AND for Incidental Underground Storage",
"PI" : "Power and Incidental Underground Storage",
"PR" : "Power",
"PS" : "Supplemental Power and Incidental Underground Storage",
"PW" : "Public Water Supply",
"RC" : "Groundwater Recharge",
"RD" : "Raise Dam (for increase in head for power production)",
"SC" : "Supplemental Cooling (an appropriation for water for cooling through a system that has a prior appropriation for cooling)",
"SD" : "Supplemental Domestic",
"SF" : "Supplemental Fish Culture",
"SI" : "Supplemental Irrigation (irrigation from reservoir on lands also covered by natural flow appropriation.)",
"SO" : "Storage Use-only (irrigation from reservoir on lands not covered by natural flow appropriation)",
"SP" : "Supplemental Power (an appropriation for water for power through a system that has a prior appropriation for power)",
"SS" : "Supplemental Storage (an appropriation that has a prior appropriation for storage)",
"ST" : "Storage",
"SU" : "Storage and Incidental Underground Storage",
"TI" : "Temporary Transfer to In-Stream Use",
"UI" : "Supplemental Irrigation and Incidental Underground Storage",
"US" : "Incidental Underground Storage",
"WS" : "Waste Storage",
"WT" : "Wetlands"}
def assignRightUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unknown'
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = NebraskaBenUseCodeDict[String1]
        except:
            outList = 'Unknown'
    return outList

df['BeneficialUseCategory'] = df.apply(lambda row: assignRightUse(row['RightUse']), axis=1)

In [9]:
df

,RightID,RightStatus,RightUse,IrrigationProject,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,ProGrant = Acres *(1/Rate),CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,Long,Lat,County,BeneficialUseCategory
0,4105,Active,IR,NaN,A-10003,NWNE S34-T2-R8E,176836,1.0,1.0,1D,NaN,Pump,NaN,Wolf Creek,2.64,CFS,1184.0,70.0,185.0,NaN,Gage,Lower Big Blue,102702020604,1961-12-21,NaN,1962-07-16T00:00:00,Open,2016-04-16T00:00:00,Open,Natural Flow (Reminder),-96.514694,40.094689,Gage,Irrigation from Natural Stream
1,6155,Active,IR,NaN,A-10009,NENE S25-T17-R7W,188876,1.0,1.0,2A,NaN,Pump,NaN,Timber Creek,0.77,CFS,345.0,70.0,54.0,NaN,Nance,Lower Loup,102100100503,1962-01-17,NaN,NaN,Open,2016-04-28T00:00:00,Open,Natural Flow (Reminder),-98.078058,41.411544,Nance,Irrigation from Natural Stream
2,7967,Active,IR,NaN,A-10013,NENE S14-T34-R36W,188035,1.0,1.0,2C,NaN,Wickman Canals,NaN,Bear Creek,0.58,CFS,260.0,70.0,40.6,NaN,Cherry,Middle Niobrara,101500031608,1962-02-13,NaN,NaN,Open,2016-04-18T00:00:00,Open,Natural Flow (Reminder),-101.534059,42.917162,Cherry,Irrigation from Natural Stream
3,2410,Active,IR,NaN,A-10014,NW S25-T5-R3W,177345,1.0,1.0,1C,NaN,Pump,NaN,"Sandy Creek, Dry, Trib. To",0.74,CFS,332.0,70.0,52.0,NaN,Fillmore,Little Blue,102702060703,1962-02-13,NaN,NaN,Open,2016-04-15T00:00:00,Open,Natural Flow (Reminder),-97.613026,40.370681,Fillmore,Irrigation from Natural Stream
4,8464,Active,SO,NaN,A-10020,NENW S4-T32-R11W,190897,1.0,1.0,2C,NaN,Witherwax Canal,NaN,Witherwax Reservoir,14.00,AF,0.0,NaN,32.1,NaN,Holt,Lower Niobrara,101500070504,1961-02-06,NaN,NaN,NaN,NaN,NaN,NaN,-98.611769,42.784548,Holt,Storage Use-only (irrigation from reservoir on...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8112,7784,Active,IR,NaN,D-980B,S25-T29-R56W,187139,1.0,1.0,2C,NaN,Mcginley-Stover N. & Cook Canal No. 2,NaN,Niobrara River,0.16,CFS,71.0,70.0,10.9,NaN,Sioux,Upper Niobrara-White,101500020401,1891-05-31,NaN,1899-11-11T00:00:00,Regulating,2018-06-16T00:00:00,Regulating,Gage Height Posting,-103.842556,42.454258,Sioux,Irrigation from Natural Stream
8113,7787,Active,IR,NaN,D-980R,SENW S1-T28-R56W,187136,1.0,1.0,2C,NaN,Pump,NaN,Niobrara River,0.52,CFS,233.0,70.0,36.1,NaN,Sioux,Upper Niobrara-White,101500020507,1891-05-31,NaN,NaN,Open,2017-09-22T00:00:00,Open,Spencer Hydropower,-103.811240,42.430200,Sioux,Irrigation from Natural Stream
8114,9099,Active,IR,NaN,D-981,S7-T33-R56W,187867,1.0,1.0,2E,NaN,Dout Brothers Canal,NaN,Jim Creek,0.64,CFS,287.0,70.0,45.0,NaN,Sioux,Upper Niobrara-White,101201080101,1889-05-15,NaN,1900-02-09T00:00:00,Close,2017-09-13T00:00:00,Close,Riparian Administration,-103.957085,42.853727,Sioux,Irrigation from Natural Stream
8115,9165,Active,IR,NaN,D-984,S22-T33-R54W,183608,1.0,1.0,2E,NaN,Homestead Canal,NaN,"Jim Creek, East, Trib. To",0.21,CFS,94.0,70.0,15.0,NaN,Sioux,Upper Niobrara-White,101201080106,1890-05-31,NaN,1899-11-04T00:00:00,NaN,NaN,NaN,NaN,-103.662795,42.813928,Sioux,Irrigation from Natural Stream


In [10]:
#Exporting to Finished File
df.to_csv('P_NebraskaMaster.csv', index=False)  # The output

PermissionError: [Errno 13] Permission denied: 'P_NebraskaMaster.csv'